In [1]:
import pandas as pd
df=pd.read_csv("../data/processed/online_retail_clean.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Month,DayOfWeek
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12,Wednesday


In [2]:
total_revenue = df["Revenue"].sum()
num_customers = df["CustomerID"].nunique()
num_orders = df["InvoiceNo"].nunique()

aov = total_revenue / num_orders

print(f"Revenue: £{total_revenue:,.0f}")
print(f"Customers: {num_customers}")
print(f"AOV: £{aov:.2f}")

Revenue: £8,911,408
Customers: 4339
AOV: £480.76


In [3]:
orders_per_customer = df.groupby("CustomerID")["InvoiceNo"].nunique()

In [4]:
# new vs repeat customers
new_customers = (orders_per_customer == 1).sum()
repeat_customers = (orders_per_customer > 1).sum()

repeat_rate = repeat_customers / orders_per_customer.shape[0]

print(f"Repeat Purchase Rate: {repeat_rate:.2%}")


Repeat Purchase Rate: 65.57%


In [5]:
# revenue contribution by funnal stage
customer_revenue = df.groupby("CustomerID")["Revenue"].sum()

repeat_customer_ids = orders_per_customer[orders_per_customer > 1].index
repeat_revenue = customer_revenue.loc[repeat_customer_ids].sum()

repeat_revenue_share = repeat_revenue / total_revenue
print(f"Repeat Customer Revenue Share: {repeat_revenue_share:.2%}")

Repeat Customer Revenue Share: 93.08%


In [6]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

In [7]:
# time to repeat the purchase
df_sorted = df.sort_values(["CustomerID", "InvoiceDate"])

df_sorted["PrevPurchaseDate"] = (
    df_sorted.groupby("CustomerID")["InvoiceDate"].shift(1)
)

df_sorted["DaysBetweenPurchases"] = (
    df_sorted["InvoiceDate"] - df_sorted["PrevPurchaseDate"]
).dt.days

In [8]:
df_sorted["DaysBetweenPurchases"].describe()

count    393585.000000
mean          1.425219
std          12.297206
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         365.000000
Name: DaysBetweenPurchases, dtype: float64

In [9]:
# monthly funnel
monthly_metrics = df.groupby("Month").agg(
    revenue=("Revenue", "sum"),
    orders=("InvoiceNo", "nunique"),
    customers=("CustomerID", "nunique")
)

monthly_metrics["AOV"] = monthly_metrics["revenue"] / monthly_metrics["orders"]
monthly_metrics.head()

,revenue,orders,customers,AOV
Month,,,,
2010-12,572713.890,1400,885,409.081350
2011-01,569445.040,987,741,576.945329
2011-02,447137.350,998,758,448.033417
2011-03,595500.760,1321,974,450.795428
2011-04,469200.361,1149,856,408.355406


In [10]:
# geographic funnel
country_metrics = (
    df.groupby("Country")
      .agg(
          revenue=("Revenue", "sum"),
          customers=("CustomerID", "nunique")
      )
      .sort_values("revenue", ascending=False)
)
country_metrics.head(5)

,revenue,customers
Country,,
United Kingdom,7308391.554,3921
Netherlands,285446.340,9
EIRE,265545.900,3
Germany,228867.140,94
France,209024.050,87


In [11]:
# product funnel
product_revenue = (
    df.groupby("StockCode")["Revenue"]
      .sum()
      .sort_values(ascending=False)
)

In [12]:
top_products = product_revenue.head(10)

In [13]:
top_products

StockCode
23843     168469.60
22423     142592.95
85123A    100603.50
85099B     85220.78
23166      81416.73
POST       77803.96
47566      68844.33
84879      56580.34
M          53779.93
23084      51346.20
Name: Revenue, dtype: float64

The analysis showed that repeat customers contribute the majority of total revenue, while time-to-second-purchase is a strong indicator of long-term value. These results highlight that improving retention and repeat engagement is more impactful than focusing solely on new customer acquisition.